# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
from azureml.core import Workspace, Experiment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice

## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
#getting data: the dataset was uploaded to and is accesible through my github repo
#ancla
from azureml.data.dataset_factory import TabularDatasetFactory

rawdata_homic2018 = "https://raw.githubusercontent.com/hftamayo/azuremlproj03/main/cad2018.csv"
dshomic2018 = TabularDatasetFactory.from_delimited_files(path=rawdata_homic2018, separator=',')
hyperpdhomic2018 = dshomic2018.to_pandas_dataframe().dropna()

In [3]:
ws = Workspace.from_config()
expname = 'hyperp3htb'

exphyper=Experiment(ws, expname)

In [4]:
# Compute name should contain only letters, digits, hyphen and should be 2-16 charachters long
ccname = "ccp03hyp"
try:
    ccluster = ComputeTarget(ws, ccname)
    print(f"{ccname} exists already")
except:
    ccconfig = AmlCompute.provisioning_configuration(vm_size="Standard_D2_V2", max_nodes=5)
    ccluster = ComputeTarget.create(ws, ccname, ccconfig)
ccluster.wait_for_completion(show_output=True)


Creating
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [5]:
# TODO: Create an early termination policy. This is not required if you are using Bayesian sampling.
early_termination_policy = BanditPolicy(evaluation_interval=1, slack_factor=0.2, delay_evaluation=5)

#TODO: Create the different params that you will be using during training
param_sampling = RandomParameterSampling({"--C": choice(1,2,3,4,5), "--max_iter": choice(80,100,120,150,170,200)})

#TODO: Create your estimator and hyperdrive config
estimator = SKLearn(source_directory='.', compute_target=ccluster, entry_script='train.py')

hyperdrive_run_config = HyperDriveConfig(estimator=estimator, policy=early_termination_policy, primary_metric_name="Accuracy",
                                          hyperparameter_sampling=param_sampling,
                                         max_total_runs=20,
                                          primary_metric_goal=PrimaryMetricGoal.MAXIMIZE)

'SKLearn' estimator is deprecated. Please use 'ScriptRunConfig' from 'azureml.core.script_run_config' with your own defined environment or the AzureML-Tutorial curated environment.


In [6]:
#TODO: Submit your experiment
hyperDrive_run = exphyper.submit(hyperdrive_run_config)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [7]:
RunDetails(hyperDrive_run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [19]:
brmodel = hyperDrive_run.get_best_run_by_primary_metric()
brmetrics = brmodel.get_metrics()

print('Best Run Id: ', brmodel.id)
print('\n Accuracy:', brmetrics['Accuracy'])
print('\n Regularization Strength:',brmetrics['Regularization Strength:'])
print('\n Max Iterations:',brmetrics['Max iterations:'])

Best Run Id:  HD_7fc67ee8-156e-40c1-b415-4c7723806da5_0

 Accuracy: 0.8914342629482072

 Regularization Strength: 4.0

 Max Iterations: 150


In [32]:
import joblib
os.makedirs('outputs', exist_ok=True)
joblib.dump(value=brmodel.id, filename='outputs/bhypermodel.joblib')

['outputs/bhypermodel.joblib']

In [34]:
#TODO: Save the best model
#ancla
model = brmodel.register_model(model_name='HyperDrive_HighAccuracy', model_path='outputs/', 
                                properties={'Accuracy': brmetrics['Accuracy'],
                                            'Regularization Strength': brmetrics['Regularization Strength:'],
                                           'Max Iterations': brmetrics['Max iterations:']})
#model = brmodel.register_model(model_name='hyperbestmodel', model_path='outputs/bhypermodel.joblib')


## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [36]:
# Download scoring file 
brmodel.download_file('outputs/scoring_file_v_1_0_0.py', 'hypscoring.py')

# Download environment file
brmodel.download_file('outputs/conda_env_v_1_0_0.yml', 'hypenviro.yml')
#from azureml.core import Environment
#from azureml.core.model import InferenceConfig
#env = Environment.get(workspace=ws, name="AzureML-AutoML")
#inference_config = InferenceConfig(entry_script='hypscoring.py', environment=env)

UserErrorException: UserErrorException:
	Message: File with path outputs/scoring_file_v_1_0_0.py was not found,
available files include: azureml-logs/55_azureml-execution-tvmps_7c467f4b7b31e8408ffe4b444fca5d7aa2dee41ebf231a829a0b72fe8b80902b_d.txt,azureml-logs/65_job_prep-tvmps_7c467f4b7b31e8408ffe4b444fca5d7aa2dee41ebf231a829a0b72fe8b80902b_d.txt,azureml-logs/70_driver_log.txt,azureml-logs/75_job_post-tvmps_7c467f4b7b31e8408ffe4b444fca5d7aa2dee41ebf231a829a0b72fe8b80902b_d.txt,azureml-logs/process_info.json,azureml-logs/process_status.json,logs/azureml/103_azureml.log,logs/azureml/job_prep_azureml.log,logs/azureml/job_release_azureml.log,outputs/hyperDrive_4.0_150.
	InnerException None
	ErrorResponse 
{
    "error": {
        "code": "UserError",
        "message": "File with path outputs/scoring_file_v_1_0_0.py was not found,\navailable files include: azureml-logs/55_azureml-execution-tvmps_7c467f4b7b31e8408ffe4b444fca5d7aa2dee41ebf231a829a0b72fe8b80902b_d.txt,azureml-logs/65_job_pre

In [ ]:
from azureml.core.environment import Environment
from azureml.core.model import InferenceConfig

infcfg = InferenceConfig(entry_script='hypscoring.py',
                                    environment=Environment.from_conda_specification(name='myenv',file_path='hypenviro.yml'))

# deploying the model via WebService
from azureml.core.webservice import AciWebservice

depcfg = AciWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 1)
webservice = Model.deploy(ws, "wshypp03ht01", [hypbmlp3ht], infcfg, depcfg)
webservice.wait_for_deployment(show_output = True)
print(webservice.state)

print(webservice.scoring_uri)

print(webservice.swagger_uri)

In [ ]:
import json

#Importing the data set for testing 
tsthomic2018 = hyperpdhomic2018.sample(10) 
lblhomic2018 = tsthomic2018.pop('sexo')

tstdatahomic2018 = json.dumps({'data': tsthomic2018.to_dict(orient='records')})

print(tstdatahomic2018)

TODO: In the cell below, send a request to the web service you deployed to test it.

In [ ]:
import requests

headers = {'Content-type': 'application/json'}

response = requests.post(webservice.scoring_uri, tstdatahomic2018, headers=headers)

In [ ]:
# Print results from the inference
print(response.text)

In [ ]:
# Print original labels
print(lblhomic2018)

TODO: In the cell below, print the logs of the web service and delete the service

In [ ]:
print(webservice.get_logs())

In [ ]:
#Optional: converting the model to ONNX format
brmodel2 = hyperDrive_run.get_output()
from azureml.automl.runtime.onnx_convert import OnnxConverter

brmodel2, onnx_model= hyperDrive_run.get_output(return_onnx_model=True)
OnnxConverter.save_onnx_model(onnx_model, file_path="./hyperbmodel.onnx")
print(brmodel2)

In [ ]:
#BEST PRACTICES: deleting webservice and CCluster that won't be used anymore
webservice.delete()
ccluster.delete()